In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = ['fetch_all_securities_list', 'fetch_snp500_securities_list']

In [2]:
# Parameters
sec_base_url = "https://data.sec.gov/submissions/CIK{cik_str}.json"
sample_pct = 1
snp_filter = False
upstream = {
    "fetch_snp500_securities_list": {
        "nb": "/root/market_watch/output/notebooks/fetch_snp500_securities_list.ipynb",
        "data": "/root/market_watch/output/data/raw/snp500_securities_list.csv",
    },
    "fetch_all_securities_list": {
        "nb": "/root/market_watch/output/notebooks/fetch_all_securities_list.ipynb",
        "data": "/root/market_watch/output/data/raw/all_securities_list.csv",
    },
}
product = {
    "nb": "/root/market_watch/output/notebooks/fetch_securities.ipynb",
    "data": "/root/market_watch/output/data/raw/securities.csv",
}


In [3]:
import pandas as pd
import json
import requests
import time 
from pathlib import Path
from tqdm import tqdm
from src.utils import preprocess_text

Get list of publicaly traded companies, ticker and associated cik ID. We will use cik ID to pull data from SEC's [DATA APIS](https://www.sec.gov/edgar/sec-api-documentation)

In [4]:
traded_company_df = pd.read_csv(upstream["fetch_all_securities_list"]["data"], index_col=0)
snp500_company_df = pd.read_csv(upstream["fetch_snp500_securities_list"]["data"], index_col=0)

# SEC data API needs cik number to be padded by 0's to make it a length of 10 string
traded_company_df['cik'] = traded_company_df.cik_str
traded_company_df.cik_str = traded_company_df.cik_str.astype(str).str.zfill(10)
snp500_company_df['cik_str'] = snp500_company_df.CIK.astype(str).str.zfill(10)

Download dataset for publicly traded companies incl. metadata like previous name etc. See the [DEVELOPER FAQ](https://www.sec.gov/os/webmaster-faq#developers) to get more infomration about rate limit and API call requirements.

In [5]:
# snp_filter parameter controls if we want to fetch all master publicly traded securities or just the snp500 ones
if snp_filter == True:
    traded_companies = snp500_company_df.cik_str
else:
    # sample_pct is helpful parameter to help speed up development by selecting only a fraction of securities out of 12,227 master securities       
    traded_companies = traded_company_df.cik_str.sample(frac=sample_pct)

Let's Fetch SEC submission data. This API has 10 request per second limit. so it will take about 1 hour to finish.preprocess_text

In [6]:
# SEC data API requires explicit setting of User-agent as email address, otehrwise we will get 404 error
headers = {
  'User-Agent': 'vgaurav@umich.edu'
}

df_lst = []
counter = 0
#  let's reuse connection pool by using same session.
session = requests.Session()

for cik_str in tqdm(traded_companies):
    url = sec_base_url.format(cik_str = cik_str)
    # we could have used following code, but it does not work due to nested json format of payload
    # pd.read_json(url, stosample_pctptions = {'User-Agent': 'vgaurav@umich.edu'})
    response = session.get(url, headers=headers)
    data = json.loads(response.text)
    df_lst.append(pd.json_normalize(data))
    counter += 1
    # SEC API restricts 10 API request per second, so we will use counter and time to meet this restriction
    if counter % 10 == 0:
        time.sleep(1)
        
securities_df = pd.concat(df_lst)

  0%|                                                                    | 0/9213 [00:00<?, ?it/s]

  0%|                                                            | 1/9213 [00:00<36:53,  4.16it/s]

  0%|                                                            | 3/9213 [00:00<23:11,  6.62it/s]

  0%|                                                            | 4/9213 [00:00<25:27,  6.03it/s]

  0%|                                                            | 5/9213 [00:00<24:57,  6.15it/s]

...

100%|████████████████████████████████████████████████████████▉| 9210/9213 [41:22<00:01,  1.91it/s]

100%|████████████████████████████████████████████████████████▉| 9211/9213 [41:22<00:00,  2.34it/s]

100%|████████████████████████████████████████████████████████▉| 9212/9213 [41:22<00:00,  2.94it/s]

100%|█████████████████████████████████████████████████████████| 9213/9213 [41:23<00:00,  3.42it/s]

100%|█████████████████████████████████████████████████████████| 9213/9213 [41:23<00:00,  3.71it/s]

In [7]:
securities_df.cik = securities_df.cik.astype(int)
securities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9213 entries, 0 to 0
Data columns (total 47 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   cik                                           9213 non-null   int64 
 1   entityType                                    9213 non-null   object
 2   sic                                           9213 non-null   object
 3   sicDescription                                9213 non-null   object
 4   insiderTransactionForOwnerExists              9213 non-null   int64 
 5   insiderTransactionForIssuerExists             9213 non-null   int64 
 6   name                                          9213 non-null   object
 7   tickers                                       9213 non-null   object
 8   exchanges                                     9213 non-null   object
 9   ein                                           9204 non-null   object
 10  des

In [8]:
traded_company_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9213 entries, 0 to 12160
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   cik_str  9213 non-null   object
 1   ticker   9213 non-null   object
 2   title    9213 non-null   object
 3   cik      9213 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 359.9+ KB


In [9]:
merged_df = securities_df.merge(traded_company_df, on='cik', how='inner')
# merged_df = merged_df.dropna(subset=['tickers'])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9213 entries, 0 to 9212
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   cik                                           9213 non-null   int64 
 1   entityType                                    9213 non-null   object
 2   sic                                           9213 non-null   object
 3   sicDescription                                9213 non-null   object
 4   insiderTransactionForOwnerExists              9213 non-null   int64 
 5   insiderTransactionForIssuerExists             9213 non-null   int64 
 6   name                                          9213 non-null   object
 7   tickers                                       9213 non-null   object
 8   exchanges                                     9213 non-null   object
 9   ein                                           9204 non-null   object
 10  

In [10]:
snp500_company_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 503
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 500 non-null    object
 1   Security               500 non-null    object
 2   SEC filings            500 non-null    object
 3   GICS Sector            500 non-null    object
 4   GICS Sub-Industry      500 non-null    object
 5   Headquarters Location  500 non-null    object
 6   Date first added       455 non-null    object
 7   CIK                    500 non-null    int64 
 8   Founded                500 non-null    object
 9   cik_str                500 non-null    object
dtypes: int64(1), object(9)
memory usage: 43.0+ KB


In [11]:
# left join ensures we populate fields from snp 500 df where applicable
merged_securities_df = merged_df.merge(snp500_company_df, left_on='cik', right_on='CIK', how='left')
merged_securities_df = merged_securities_df[['cik','ticker', 'exchanges', 'ein', 'name','formerNames', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'sic', 'sicDescription', 'Headquarters Location']]
merged_securities_df.columns = ['cik','ticker', 'exchanges', 'ein', 'full_name','former_names', 'short_name', 'gics_sector', 'gics_sub_industry', 'sic','sic_description', 'headquarters_location']
merged_securities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9213 entries, 0 to 9212
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   cik                    9213 non-null   int64 
 1   ticker                 9213 non-null   object
 2   exchanges              9213 non-null   object
 3   ein                    9204 non-null   object
 4   full_name              9213 non-null   object
 5   former_names           9213 non-null   object
 6   short_name             499 non-null    object
 7   gics_sector            499 non-null    object
 8   gics_sub_industry      499 non-null    object
 9   sic                    9213 non-null   object
 10  sic_description        9213 non-null   object
 11  headquarters_location  499 non-null    object
dtypes: int64(1), object(11)
memory usage: 935.7+ KB


In [12]:
output_file_path = product['data']
print(len(merged_securities_df))

9213


In [13]:
merged_securities_df.to_csv(output_file_path)
print(f"Saved file {output_file_path}")

Saved file /root/market_watch/output/data/raw/securities.csv


In [14]:
merged_securities_df.sample(10)

,cik,ticker,exchanges,ein,full_name,former_names,short_name,gics_sector,gics_sub_industry,sic,sic_description,headquarters_location
5071,885462,GURE,[Nasdaq],133637458,"GULF RESOURCES, INC.","[{'name': 'DIVERSIFAX INC', 'from': '1996-10-1...",NaN,NaN,NaN,2800,Chemicals & Allied Products,NaN
8569,1818089,BOAC.U,"[, NYSE, NYSE]",000000000,Bluescape Opportunities Acquisition Corp.,[],NaN,NaN,NaN,6770,Blank Checks,NaN
1964,1872195,BULL,[],000000000,Bullish,[],NaN,NaN,NaN,6199,Finance Services,NaN
601,1680139,HLYK,[OTC],471634127,HealthLynked Corp,[],NaN,NaN,NaN,8011,Services-Offices & Clinics of Doctors of Med...,NaN
3329,1745916,PFSI,[NYSE],831098934,"PennyMac Financial Services, Inc.","[{'name': 'New PennyMac Financial Services, In...",NaN,NaN,NaN,6162,Mortgage Bankers & Loan Correspondents,NaN
501,764622,PNW,[NYSE],860512431,PINNACLE WEST CAPITAL CORP,[],Pinnacle West,Utilities,Multi-Utilities,4911,Electric Services,"Phoenix, Arizona"
314,1500620,BGAVF,[OTC],000000000,Bravada Gold Corp,[],NaN,NaN,NaN,,,NaN
5874,1819559,WDI,[NYSE],000000000,Western Asset Diversified Income Fund,[],NaN,NaN,NaN,,,NaN
3050,31462,ECL,[NYSE],410231510,ECOLAB INC.,"[{'name': 'ECOLAB INC', 'from': '1994-02-10T00...",Ecolab,Materials,Specialty Chemicals,2840,"Soap, Detergents, Cleang Preparations, Perfume...","Saint Paul, Minnesota"
8624,1628040,RCIAX,[Nasdaq],472522233,Alternative Credit Income Fund,"[{'name': 'Resource Credit Income Fund', 'from...",NaN,NaN,NaN,,,NaN


In [15]:
del securities_df, traded_company_df, traded_companies, snp500_company_df, merged_securities_df